## Problem 2: A charge with a burst of sudden acceleration

A charge is initially at rest at the origin. At t=0 the charge starts to move along the x direction with constant acceleration. The acceleration lasts for 1 ns and the charge reaches a speed of 0.5c. Afterwards the charge moves with the constant speed of 0.5c. Plot out the electric field of the charge on the xz plane at different times and make an animation. Is your finding consistent with your expectation?